In [70]:
import requests
import time

In [71]:
Token = "f50552091d9148249640201d2dc3e31f"

## Upload Audio File (wav)

In [72]:
# filename = "/Users/cv0361/src/Hackathon/Audio/DallasVideo.wav"
# filename = "/Users/cv0361/src/Hackathon/Audio/Dallas.wav"
filename = "/Users/cv0361/src/Hackathon/AARecording.wav"

def read_file(filename, chunk_size=5242880):
    with open(filename, 'rb') as _file:
        while True:
            data = _file.read(chunk_size)
            if not data:
                break
            yield data

headers = {'authorization': f"{Token}"}

response = requests.post('https://api.assemblyai.com/v2/upload',
                        headers=headers,
                        data=read_file(filename))

In [73]:
resobj = response.json()
# resobj

In [74]:
upload_url = resobj['upload_url']
upload_url

'https://cdn.assemblyai.com/upload/6673a93c-8194-4114-9a67-54e1eb229574'

## Transcribe Audio

In [75]:
json = {
    "audio_url": f"{upload_url}",
    "punctuate": True,
    "format_text": False
}

headers = {
    "authorization": f"{Token}",
    "content-type": "application/json"
}

response = requests.post("https://api.assemblyai.com/v2/transcript", json=json, headers=headers)

In [76]:
resobj = response.json()
# resobj

In [77]:
transcript_id = resobj['id']
transcript_id

'r7lqobi024-763f-476d-a2d0-51732b10d212'

## Wait for completion and Transcription to be Available

In [78]:
loop = 0
MAX_COUNT = 30
wait_sec = 1
status = ""

while (status != "completed") and (loop < MAX_COUNT):
    loop += 1
    time.sleep(wait_sec)
    
    # Get Transcript
    endpoint = f"https://api.assemblyai.com/v2/transcript/{transcript_id}"
    headers = {"authorization": f"{Token}"}
    response = requests.get(endpoint, headers=headers)
    
    resobj = response.json()
    status = resobj['status']
    # End 
    
    print("Waiting....", loop)
    
if loop < MAX_COUNT:
    print("API done!")
    text = resobj['text']
else:
    print("Timed out!")

Waiting.... 1
Waiting.... 2
Waiting.... 3
Waiting.... 4
Waiting.... 5
API done!


In [79]:
text

'Tell me something about Tokyo.'

In [80]:
def getTextWithConfidence(WordList, threshold=0.5):
    strPhrase = ""
    
    for word in WordList:
        if word['confidence'] > threshold:
            strPhrase += word['text'] + " "

    return strPhrase
            
getTextWithConfidence(resobj['words'])

'Tell me something about Tokyo. '

In [81]:
resobj

{'id': 'r7lqobi024-763f-476d-a2d0-51732b10d212',
 'language_model': 'assemblyai_default',
 'acoustic_model': 'assemblyai_default',
 'language_code': 'en_us',
 'status': 'completed',
 'audio_url': 'https://cdn.assemblyai.com/upload/6673a93c-8194-4114-9a67-54e1eb229574',
 'text': 'Tell me something about Tokyo.',
 'words': [{'text': 'Tell',
   'start': 810,
   'end': 974,
   'confidence': 0.99989,
   'speaker': None},
  {'text': 'me',
   'start': 1012,
   'end': 1166,
   'confidence': 0.99987,
   'speaker': None},
  {'text': 'something',
   'start': 1188,
   'end': 1374,
   'confidence': 0.99943,
   'speaker': None},
  {'text': 'about',
   'start': 1412,
   'end': 1614,
   'confidence': 0.9996,
   'speaker': None},
  {'text': 'Tokyo.',
   'start': 1652,
   'end': 2090,
   'confidence': 0.98948,
   'speaker': None}],
 'utterances': None,
 'confidence': 0.997654,
 'audio_duration': 3,
 'punctuate': True,
 'format_text': False,
 'dual_channel': None,
 'webhook_url': None,
 'webhook_status_c